# Import/Export-Adjusted Green House Gas Emissions
## Final Assignment for EPA1333 Computer Engineering for Scientific Computing

Authors:

Patrick Steinmann #4623991

Stefan Wigman     #4016246


## Abstract

High-pollutant industrial processes often take place in developing countries, the resulting products often being exported to developed countries. We analyze this "offshoring" of green house gas (GHG) emissions by considering country-to-country import/export balances and national GHG emissions. We attempt to assign each country each it's "true" GHG emissions by determining which emissions that country causes in other countries, and then attributing these "offshored" emissions accordingly. We find that #TODO

## Introduction

In partial fulfillment of the course requirements of EPA1333, we were tasked to conduct an original and non-trivial data analysis related to climate change.

We chose to investigate the phenomenon of "outsourcing" green house gas (GHG) emissions. Many emission-intensive activities take place in countries with poor emissions records - however, these countries often export the products of these activities to countries with much better emissions records. In essence, the emissions are being outsourced. A trivial example is the import of electrical energy - highly polluting coal is burned in a power plant in poorly developed country A, and the generated energy is exported to highly developed country B. Country B can claim low GHG emissions - after all, the coal is being burned in A, which, as a poorly developed country, has much more leeway regarding pollution.

Our research question therefore is as follows:

*When considering a country's import/export-adjusted emissions, does this differ significantly from their claimed emissions records, and how has this developed over time?*

## Methodology

### Approach

We tackled our research question by first finding, importing and cleaning import/export data between countries. Specifically, we were interested in total import/export (that is, goods and services) from and to each country, for our time range of interest. We defined this time range as 1995 to 2015, giving us a long enough 20-year interval while staying largely inside the time range where useful data was available.

We then obtained data on every country's GDP and GHG emissions. These emissions are reported as total GHG emitted over a year in a country, irrespective of use/destination.

$ emissions_{nominal} = emissions_{self} + emissions_{export} $

By comparing export volume and GDP, we were able to determine which percentage of a country's GHG emissions are "self-caused", and which are "offshored" - that is, emissions created by products destined to by exported. In essence, these emissions are the fault of the country importing those products, not the emittant's.

We could then assign each country, based on its imports, a percentage of their import partners' GHG emissions, thus arriving at each country's import/export adjusted (or "true") emissions.

$ emissions_{true} = emissions_{self} + emissions_{import} $

### Assumptions & Simplifications

* countries export a broadly similar product palette to every export partner
* no re-export or re-import

## Results/Work

## Analysis

## Conclusion

## Reflection

## References & Sources